In [9]:
%pip install ultralytics


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install kagglehub


Note: you may need to restart the kernel to use updated packages.


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fatihkgg/affectnet-yolo-format")

print("Path to dataset files:", path)

100%|██████████| 258M/258M [00:11<00:00, 22.9MB/s] 

Extracting files...


Path to dataset files: /home/sagemaker-user/.cache/kagglehub/datasets/fatihkgg/affectnet-yolo-format/versions/2


In [6]:
%cp -r /home/sagemaker-user/.cache/kagglehub/datasets/fatihkgg/affectnet-yolo-format/versions/2/YOLO_format .

In [ ]:
from ultralytics import YOLO

# ---------------- Hyper‑parameters -----------------
EPOCHS = 500
BATCH  = 32
IMGSZ  = 96
DEVICE = '0'   # GPU id   (use 'cpu' to force CPU)
# ---------------------------------------------------

# model = YOLO('yolo11n.pt')   # base nano model
model = YOLO("./runs/train/affectnet_yolov11/weights/last.pt")


In [2]:
results = model.train(
    data='./YOLO_format/data.yaml',
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=DEVICE,
    workers=8,
    project='runs/train',
    name='affectnet_yolov11'
    ,resume=True
)

print('Best model checkpoint ->', results.best)

Ultralytics 8.3.111 🚀 Python-3.12.9 torch-2.5.1 CUDA:0 (Tesla T4, 14918MiB)
engine/trainer: task=detect, mode=train, model=runs/train/affectnet_yolov113/weights/last.pt, data=./YOLO_format/data.yaml, epochs=500, time=None, patience=100, batch=32, imgsz=96, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs/train, name=affectnet_yolov113, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=runs/train/affectnet_yolov113/weights/last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False

E0000 00:00:1745143306.414419   11557 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745143306.420153   11557 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

train: Scanning /home/sagemaker-user/YOLO_format/train/labels.cache... 17101 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17101/17101 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 173.7±40.0 MB/s, size: 5.1 KB)


...

                   all       5406       5406      0.704      0.744      0.783      0.783



25 epochs completed in 0.399 hours.
Optimizer stripped from runs/train/affectnet_yolov113/weights/last.pt, 5.5MB
Optimizer stripped from runs/train/affectnet_yolov113/weights/best.pt, 5.5MB

Validating runs/train/affectnet_yolov113/weights/best.pt...
Ultralytics 8.3.111 🚀 Python-3.12.9 torch-2.5.1 CUDA:0 (Tesla T4, 14918MiB)
YOLO11n summary (fused): 100 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 85/85 [00:16<00:00,  5.28it/s]


                   all       5406       5406      0.699      0.746      0.792      0.792
                 Anger        712        712      0.629      0.794      0.783      0.783
              Contempt        618        618      0.687      0.793      0.827      0.827
               Disgust        672        672      0.682      0.695      0.764      0.764
                  Fear        622        622      0.744      0.698      0.797      0.797
                 Happy        791        791      0.843       0.88       0.94       0.94
               Neutral        514        514      0.634       0.64      0.675      0.675
                   Sad        603        603      0.674      0.657      0.724      0.724
              Surprise        874        874      0.701      0.814      0.824      0.824
Speed: 0.0ms preprocess, 0.2ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/train/affectnet_yolov113
MLflow: results logged to runs/mlflow
MLflow: disable with 'yolo setti

AttributeError: 'DetMetrics' object has no attribute 'best'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP).

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class.
        names (dict): A dictionary of class names.
        box (Metric): An instance of the Metric class for storing detection results.
        speed (dict): A dictionary for storing execution times of different parts of the detection process.
        task (str): The task type, set to 'detect'.
    